# FASE 3: Análisis de Valor (Rubén) Coordinador
## Este notebook realiza el análisis de sentimiento y de inteligencia de usuarios.
### Rubén - Análisis de Sentimiento

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from collections import Counter
import os
import json

# Cargar datos
df = pd.read_csv('../data/processed/dataset_clean.csv')
print(f"Datos cargados: {len(df)} reseñas")


Datos cargados: 110 reseñas


In [2]:
def analyze_sentiment(text):
    analysis = TextBlob(str(text))
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity
    if polarity > 0.1: sentiment = 'Positivo'
    elif polarity < -0.1: sentiment = 'Negativo'
    else: sentiment = 'Neutral'
    return polarity, subjectivity, sentiment

print("Analizando sentimiento...")
df_sentiment = df.copy()
res = df_sentiment['texto_limpio'].apply(analyze_sentiment)
df_sentiment[['polarity', 'subjectivity', 'sentiment']] = pd.DataFrame(res.tolist(), index=df.index)


Analizando sentimiento...


In [3]:
emotion_lexicon = {
    'positivas': {'bien', 'bueno', 'excelente', 'perfecto', 'genial', 'fantastico', 'recomiendo'},
    'negativas': {'mal', 'malo', 'horrible', 'terrible', 'problema', 'error', 'queja'}
}
def detect_emotions(text):
    text_lower = str(text).lower()
    counts = {cat: sum(1 for w in words if w in text_lower) for cat, words in emotion_lexicon.items()}
    if max(counts.values()) == 0: return 'neutral'
    return max(counts, key=counts.get)

df_sentiment['emocion_predominante'] = df_sentiment['texto_limpio'].apply(detect_emotions)
df_sentiment['longitud_texto'] = df_sentiment['texto_limpio'].fillna('').str.len()
df_sentiment['num_palabras'] = df_sentiment['texto_limpio'].apply(lambda x: len(str(x).split()))


In [4]:
print("Analizando inteligencia de usuarios...")
if 'num_resenas_usuario_total' in df_sentiment.columns:
    bins = [0, 1, 3, 10, 1000]
    labels = ['Nuevo (1)', 'Casual (2-3)', 'Frecuente (4-10)', 'Experto (>10)']
    df_sentiment['segmento_usuario'] = pd.cut(df_sentiment['num_resenas_usuario_total'], bins=bins, labels=labels, include_lowest=True)

print("✓ Análisis de usuarios completado.")


Analizando inteligencia de usuarios...
✓ Análisis de usuarios completado.


In [5]:
output_path = '../data/processed/reviews_with_sentiment.csv'
df_sentiment.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✓ Dataset guardado: {output_path}")

# Word frequency
all_tokens = ' '.join(df_sentiment['texto_limpio'].dropna().astype(str)).split()
word_freq = Counter(all_tokens)
pd.DataFrame(word_freq.most_common(50), columns=['palabra', 'frecuencia']).to_csv('../data/processed/top_palabras.csv', index=False)


✓ Dataset guardado: ../data/processed/reviews_with_sentiment.csv
